In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from faker import Faker
from sqlalchemy import (
    create_engine,
    text,
    Column,
    Integer,
    String,
    Float,
    DateTime,
    ForeignKey,
    Enum as SQLEnum,
)
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import declarative_base
import enum
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

## Applicatio DB setup


In [ ]:
# --- 1) MODEL DEFINITIONS ---
Base = declarative_base()


class SubscriptionStatus(enum.Enum):
    ACTIVE = "active"
    CANCELLED = "cancelled"
    EXPIRED = "expired"


class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True, nullable=False)
    email = Column(String, unique=True, index=True, nullable=False)
    created_at = Column(DateTime, default=datetime.now(timezone.utc))


class Subscription(Base):
    __tablename__ = "subscriptions"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, unique=True, nullable=False)
    price = Column(Float, nullable=False)
    description = Column(String)


class UserSubscription(Base):
    __tablename__ = "user_subscriptions"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    subscription_id = Column(Integer, ForeignKey("subscriptions.id"), nullable=False)
    start_date = Column(DateTime, default=datetime.now(timezone.utc))
    end_date = Column(DateTime, nullable=True)
    status = Column(
        SQLEnum(SubscriptionStatus), default=SubscriptionStatus.ACTIVE, nullable=False
    )


class Invoice(Base):
    __tablename__ = "invoices"
    id = Column(Integer, primary_key=True, index=True)
    user_subscription_id = Column(
        Integer, ForeignKey("user_subscriptions.id"), nullable=False
    )
    invoice_date = Column(DateTime, nullable=False)
    amount = Column(Float, nullable=False)
    status = Column(String, nullable=False, default="paid")
    created_at = Column(DateTime, default=datetime.now(timezone.utc))

In [7]:
# --- 2) DATABASE SETUP ---
DATABASE_URL = os.getenv("SUBSCRIPTION_DB_URL", "sqlite:///./test.db")
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
Base.metadata.create_all(engine)

In [ ]:
# --- 3) RESET FUNCTION ---
def reset_database(sess):
    """
    Truncate all tables to start fresh.
    """
    sess.execute(
        text("""
        TRUNCATE TABLE
          invoices,
          user_subscriptions,
          subscriptions,
          users
        RESTART IDENTITY CASCADE
    """)
    )
    sess.commit()

In [22]:
# --- 4) SEED FUNCTIONS ---
fake = Faker()


def seed_users(sess, n_users=50):
    users = [
        User(username=fake.unique.user_name(), email=fake.unique.email())
        for _ in range(n_users)
    ]
    sess.add_all(users)
    sess.commit()
    return sess.query(User).all()


def seed_subscriptions(sess, plans):
    """
    plans: list of tuples [(name, price, description), ...]
    """
    subs = [
        Subscription(name=name, price=price, description=desc)
        for name, price, desc in plans
    ]
    sess.add_all(subs)
    sess.commit()
    return sess.query(Subscription).all()


def seed_user_subscriptions(sess, users, subs, max_per_user=2):
    """
    Randomly assign subscriptions to users.
    Each user can have between 1 and max_per_user subscriptions.
    """
    records = []
    for user in users:
        count = np.random.randint(1, max_per_user + 1)
        chosen = np.random.choice(subs, size=count, replace=False)
        for plan in chosen:
            start = fake.date_between(start_date="-2y", end_date="today")
            ended = fake.boolean(chance_of_getting_true=30)
            end = (
                fake.date_between(start_date=start, end_date="today") if ended else None
            )
            status = SubscriptionStatus.EXPIRED if ended else SubscriptionStatus.ACTIVE
            records.append(
                UserSubscription(
                    user_id=user.id,
                    subscription_id=plan.id,
                    start_date=start,
                    end_date=end,
                    status=status,
                )
            )
    sess.add_all(records)
    sess.commit()
    return sess.query(UserSubscription).all()

In [23]:
# --- 5) INVOICE GENERATION ---
def generate_invoices_df(
    engine, unpaid_pct=0.05, months_before=24, months_after=0, seed=42
):
    """
    Generate a DataFrame of invoices based on user subscriptions.
    Parameters:
    - engine: SQLAlchemy engine connected to the database.
    - unpaid_pct: Percentage of invoices that are unpaid.
    - months_before: How many months before today to consider for billing.
    - months_after: How many months after today to consider for billing.
    - seed: Random seed for reproducibility.
    Returns:
    - DataFrame with columns: user_subscription_id, invoice_date, amount, status.
    """

    np.random.seed(seed)
    df = pd.read_sql(
        """
        SELECT us.id AS user_subscription_id,
               us.start_date, us.end_date, s.price
        FROM user_subscriptions us
        JOIN subscriptions s ON us.subscription_id = s.id
    """,
        engine,
    )
    today = datetime.utcnow().date()
    invoices = []
    for _, row in df.iterrows():
        start = pd.to_datetime(row.start_date).date()
        end = (
            pd.to_datetime(row.end_date).date()
            if pd.notnull(row.end_date)
            else today + relativedelta(years=1)
        )
        win_start = today - relativedelta(months=months_before)
        win_end = today + relativedelta(months=months_after)
        bill_start = max(start, win_start)
        bill_end = min(end, win_end)
        for inv_date in pd.date_range(bill_start.replace(day=1), bill_end, freq="MS"):
            status = "unpaid" if np.random.rand() < unpaid_pct else "paid"
            invoices.append(
                {
                    "user_subscription_id": row.user_subscription_id,
                    "invoice_date": inv_date,
                    "amount": row.price,
                    "status": status,
                }
            )
    return pd.DataFrame(invoices)

In [24]:
# --- 6) ORCHESTRATOR ---
def full_reset_and_seed(n_users, plan_list, max_subs_per_user=2):
    """
    1) Clears all demo data
    2) Seeds users, subscriptions, user_subscriptions
    3) Generates & pushes invoices
    """
    # 1. Reset
    reset_database(session)

    # 2. Seed core tables
    users = seed_users(session, n_users)
    subs = seed_subscriptions(session, plan_list)
    _ = seed_user_subscriptions(session, users, subs, max_subs_per_user)

    # 3. Build & insert invoices
    invoices_df = generate_invoices_df(engine)
    invoices_df.to_sql("invoices", engine, if_exists="append", index=False)

    print(
        f"Seed complete: {n_users} users, "
        f"{len(subs)} plans, "
        f"{len(users) * max_subs_per_user} user_subscriptions, "
        f"{len(invoices_df)} invoices "
        f"({invoices_df.status.eq('unpaid').mean():.1%} unpaid)."
    )

In [25]:
plans = [
    ("Basic SD", 5.99, "Standard Definition streaming, 1 screen"),
    ("Standard HD", 9.99, "High Definition streaming, 2 screens"),
    ("Premium 4K", 15.99, "Ultra HD/4K streaming, 4 screens"),
    ("Family Plan", 19.99, "Family plan with multiple profiles"),
    ("Annual Plan", 99.99, "Annual subscription with a discount"),
]

In [26]:
full_reset_and_seed(n_users=625, plan_list=plans, max_subs_per_user=3)

/var/folders/d0/vt1dssys28dd4j2wlygf9rbr0000gn/T/ipykernel_31301/1932753658.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


Seed complete: 625 users, 5 plans, 1875 user_subscriptions, 13329 invoices (5.0% unpaid).
